# Inferencia del Modelo de Churn

## Inferencia con el modelo Campeón

Con los Modelos en Unity Catalog, pueden ser cargados para su uso en pipelines de inferencia por lotes. Las predicciones generadas pueden utilizarse para crear estrategias de retención de clientes o para análisis. El modelo en uso es el modelo __Campeón__, y vamos a cargarlo para usarlo en nuestro pipeline.

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/mlops/mlops-uc-end2end-5-v2.png?raw=true" width="1200">

<!-- Recopilar datos de uso (visualización). Elimine para deshabilitar la recopilación o desactive el rastreador durante la instalación. Consulte el README para más detalles.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=data-science&org_id=1444828305810485&notebook=%2F01-mlops-quickstart%2F05_batch_inference&demo_name=mlops-end2end&event=VIEW&path=%2F_dbdemos%2Fdata-science%2Fmlops-end2end%2F01-mlops-quickstart%2F05_batch_inference&version=1&user_hash=f7ea13a45c991650d8df810431c3e0e2b12887e9ed7e206ee8fb6209bdb2ae82">

### Se ha creado un cluster para esta demostración
Para ejecutar esta demostración, simplemente selecciona el cluster `dbdemos-mlops-end2end` en el menú desplegable ([abrir configuración del clúster](https://e2-demo-field-eng.cloud.databricks.com/#setting/clusters/1013-181446-g00pu2bj/configuration)). <br />
*Nota: Si el cluster fue eliminado después de 30 días, puedes volver a crearlo con `dbdemos.create_cluster('mlops-end2end')` o reinstalar la demo: `dbdemos.install('mlops-end2end')`*

Último ambiente testado:

mlflow==3.3.0

In [0]:
%pip install --quiet mlflow --upgrade


%restart_python

In [0]:
%run ../_resources/00-setup $setup_inference_data=true

## Desplegando el modelo para batch inference

<!--img style="float: right; margin-left: 20px" width="600" src="https://github.com/QuentinAmbard/databricks-demo/raw/main/retail/resources/images/churn_batch_inference.gif" /-->

Ahora que nuestro modelo está disponible en el Unity Catalog Model Registry, podemos cargarlo para calcular nuestras inferencias y guardarlas en una tabla para comenzar a construir dashboards.

Usaremos la función de MLflow para cargar un UDF de pyspark y distribuir nuestra inferencia en todo el clúster. Podemos cargar el modelo con Python puro y usar un DataFrame de Pandas si los datos son pequeños.

Si no sabes cómo empezar, puedes obtener un código de ejemplo desde la página de __"Artifacts"__ del experimento del modelo.

## Ejecutar inferencias

### Primero, reinstalemos los requisitos del modelo

In [0]:
from mlflow.store.artifact.models_artifact_repo import ModelsArtifactRepository


requirements_path = ModelsArtifactRepository(f"models:/{catalog}.{db}.mlops_churn@Champion").download_artifacts(artifact_path="requirements.txt") # download model from remote registry

In [0]:
%pip install --quiet -r $requirements_path


%restart_python

In [0]:
%run ../_resources/00-setup

### Batch Inference en el modelo Champion

Estamos listos para ejecutar la inferencia en el modelo Champion. Cargaremos el modelo como un UDF de Spark y generaremos predicciones para nuestros registros de clientes.

Para simplificar, asumimos que las características ya han sido extraídas para los nuevos registros de clientes y almacenadas en la tabla de características. Normalmente, esto se realiza mediante pipelines de ingeniería de características separados.

In [0]:
import mlflow


# Load customer features to be scored
inference_df = spark.read.table(f"mlops_churn_inference")
# Load champion model as a Spark UDF. You can use virtual env manager for the demo to avoid version conflict (you can remove the pip install above with virtual env)
champion_model = mlflow.pyfunc.spark_udf(spark, model_uri=f"models:/{catalog}.{db}.mlops_churn@Champion") #Use env_manager="virtualenv" to recreate a venv with the same python version if needed

# Batch score
preds_df = inference_df.withColumn('predictions', champion_model(*champion_model.metadata.get_input_schema().input_names()))

display(preds_df)


¡Eso es todo! Ahora nuestros datos pueden ser guardados como una tabla y reutilizados por el equipo de Análisis de Datos / Marketing para tomar acciones especiales y reducir el riesgo de Churn en estos clientes.

¡Tus datos también estarán disponibles en Genie para responder cualquier pregunta relacionada con churn usando texto en inglés sencillo!

### Conclusión

¡Esto es todo para la demostración rápida! Hemos revisado los conceptos básicos de MLOps y cómo Databricks te ayuda a lograrlos. Estos incluyen:

- Ingeniería de características y almacenamiento de tablas de características con etiquetas en Databricks
- AutoML, entrenamiento de modelos y seguimiento de experimentos en MLflow
- Registro de modelos como Modelos en Unity Catalog para uso gobernado
- Validación de modelos, pruebas Champion-Challenger y promoción de modelos
- Batch Inference cargando el modelo como un UDF de PySpark

Esperamos que hayas disfrutado esta demostración. Como siguiente paso, busca nuestra demostración avanzada de MLOps de extremo a extremo, que incluirá recorridos más detallados sobre los siguientes aspectos de MLOps:

- Servir características y Feature Store
- Monitoreo de datos y modelos
- Despliegue para inferencia en tiempo real